In [1]:
!pip install yfinance
!pip install yahooquery
!pip install tvdatafeed
!pip install tensorflow
from tvDatafeed import TvDatafeed, Interval
from yahooquery import Screener
import yfinance as yf   
import pandas as pd
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import random 
from tensorflow.keras.models import load_model
import os
import shutil
from datetime import datetime
import glob

In [2]:
def work_with_dir():
  if os.path.exists("/content/data/"):
    shutil.rmtree("/content/data/", ignore_errors=True)
    print("Data Folder Removed")
    os.mkdir("/content/data/")
  if not os.path.exists("/content/data/"):
    os.mkdir("/content/data/")
  if not os.path.exists("/content/extracted/"):
    os.mkdir("/content/extracted/")
def get_crypto_syms():
   # 'all_cryptocurrencies_au','all_cryptocurrencies_ca','all_cryptocurrencies_eu','all_cryptocurrencies_gb','all_cryptocurrencies_in',
   screens = [
       'all_cryptocurrencies_us', 'all_cryptocurrencies_au', 'all_cryptocurrencies_ca', 'all_cryptocurrencies_eu', 'all_cryptocurrencies_gb', 'all_cryptocurrencies_in', ]
   s = Screener()
   symbols = []
   for i in screens:
      data = s.get_screeners(i, count=250)
      dicts = data[i]['quotes']
      syms = [d['symbol'] for d in dicts]
      for sym in syms:
        symbols.append(sym)
   # print(len(symbols))
   # pieces = 15
   # new_arrays = np.array_split(symbols, pieces)
   return symbols
def spliting(data):
  X = data.drop(["suggestion"], axis=1)
  y = data[["suggestion"]]
  y = pd.get_dummies(y)
  xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size=0.2,random_state=100)
  print(xTrain.shape, end=" ")
  print(yTrain.shape)
  print(xTest.shape, end=" ")
  print(yTest.shape)
  return xTrain, xTest, yTrain, yTest
def download_data(symbols,periodd,intervall):
  indexx = 100
  work_with_dir()
  for symbol in symbols:
    if ((symbols.index(symbol)+1) % 100 == 0):
      print(f" -- {indexx}",end="")
      indexx = indexx + 100
    try:
        data = yf.download(symbol, period=periodd,interval=intervall, progress=False,show_errors=False)
        if data.empty:
           pass
        else:
             data.to_csv(f"/content/data/{symbol}.csv")         
    except:
       print("Error!")
  print(" ")
  print(f"Files In Data : {len(os.listdir('/content/data/'))}")
def each_file_proc(files,now,index):
     data = []
     unattached_dfs = []
     files = list(files)
     for file in files:
        print(f"{files.index(file)+1+index}",end=" ")
        if (files.index(file)+index+1) % 40 == 0:
          print(" ")
        address = f"/content/data/{file}"
        try:
            unattached_dfs.append(process(pd.read_csv(address)))
        except:
          pass
     nud = []
     for i in unattached_dfs:
       if i.size != 0:
         nud.append(i)
     data = np.concatenate(nud)
     unattached_dfs = []
     data = pd.DataFrame(data, columns=clmns)
     right_now = datetime.now().strftime("%H%M%S%f")
     data.to_csv(f"/content/extracted/{now}/{right_now}.csv")  
def extract_data(pieces):
  print(f"Files In Data : {len(os.listdir('/content/data/'))}")
  pd.options.mode.chained_assignment = None
  files = os.listdir("/content/data/")
  new_files = np.array_split(files, pieces)
  print("Processing File:")
  now = datetime.now().strftime("%H%M%S")
  os.mkdir(f"/content/extracted/{now}/")
  
  index = 0 
  for files in new_files:
     
     each_file_proc(files,now,index)
     index = index + len(files)
  print(" ")
  return now
def delete_all_csv(now):
   path = f'/content/extracted/{now}/*.csv'
   files = glob.glob(path)
   for file in files:
       os.remove(file)
def make_df(now):
   path = f'/content/extracted/{now}/*.parquet'
   files = glob.glob(path)
   #data = pd.DataFrame()
   data = pd.DataFrame()
   for adr in files:
     data =pd.concat([data,pd.read_parquet(adr)])
   if "Unnamed: 0" in data:
     data.drop("Unnamed: 0",axis=1,inplace=True)
   print(data.shape)
   xTrain,xTest,yTrain,yTest = spliting(data)
   data.to_parquet(f'/content/extracted/{now}/data.parquet')
   delete_all_csv(now)
   data = []
   return xTrain,xTest,yTrain,yTest
def to_par(now,howmanyfiles): 
    files = os.listdir(f"/content/extracted/{now}/")
    addresses = []
    for file in files:
      addresses.append(f"/content/extracted/{now}/{file}")
    new_adr = np.array_split(addresses,howmanyfiles)
    for adrs in new_adr:
      datas = []
      for adr in adrs:
        datas.append(pd.read_csv(adr))
      rnow = datetime.now().strftime("%H%M%S%f")
      datas = pd.concat(datas)
      datas.to_parquet(f"/content/extracted/{now}/part_{rnow}.parquet")     

In [190]:
symbols = get_crypto_syms()
print(f"Symbols : {len(symbols)}")
download_data(symbols,"max","1wk")

Symbols : 1500
Data Folder Removed
 -- 100 -- 200 -- 300 -- 400 -- 500 -- 600 -- 700 -- 800 -- 900 -- 1000 -- 1100 -- 1200 -- 1300 -- 1400 -- 1500 
Files In Data : 1500


In [51]:
clmns = list(range(1, 61))
clmns.append("suggestion")
def scaler(row):
    scaler = MinMaxScaler(feature_range=(-10,10))
    row = scaler.fit_transform(row)
    return row
def make_y(data, i):
  sugg = ""
  
  meann = 0
  for x in range(i-10, i):
      meann += data[x][1] - data[x][2]

  high2 = 0
  low2 = data[i][0] * 10000
  for x in range(i, i+10):
        if data[x][1] > high2:
            high2 = data[x][1]
        if data[x][2] < low2:
            low2 = data[x][2]

  diff = meann / 5
  mean_of_last = (data[i-1][1] + data[i-1][2])/2
  if high2 > mean_of_last + diff:
      sugg = "buy"
  elif low2 < mean_of_last - diff:
      sugg = "sell"
  else:
      sugg = "hold"
  return sugg

def process(data):
    data = data.dropna()
    row = []
    if len(data.columns) == 7:
      data = data.iloc[:, 1:]
    data = np.array(data)
    for i in range(31, data.shape[0]-10):
        grow = []
        ggggrow = []
        for x in range(1, 31):
            grow.append([data[i-x][0] - data[i-(1+x)][0]])
            ggggrow.append([data[i-x][3] - data[i-(1+x)][3]])
        arr   = np.array(grow).flatten()
        arr4  = np.array(ggggrow).flatten()
        sugg = make_y(data,i)
        arr = np.concatenate((arr,arr4), axis=0).reshape(-1,1)
        arr = scaler(arr.reshape(-1, 1))
        arr = np.append(arr, sugg)
        row.append(arr)
    return np.array(row)

In [52]:
folder_name = extract_data(100)
to_par(folder_name,10)
xTrain,xTest,yTrain,yTest = make_df(folder_name)

Files In Data : 1500
Processing File:
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40  
41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80  
81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120  
121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160  
161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200  
201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240  
241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 26

In [54]:
yTrain.suggestion_buy.value_counts()

0    95488
1    69852
Name: suggestion_buy, dtype: int64

In [55]:
model = Sequential()

model.add(Dense(512, activation='relu', input_shape=(xTrain.shape[1],)))
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(3, activation='sigmoid'))

opt = tf.keras.optimizers.Adamax()

model.compile(optimizer=opt, loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_40 (Dense)            (None, 512)               31232     
                                                                 
 dense_41 (Dense)            (None, 512)               262656    
                                                                 
 dense_42 (Dense)            (None, 512)               262656    
                                                                 
 dense_43 (Dense)            (None, 512)               262656    
                                                                 
 dense_44 (Dense)            (None, 3)                 1539      
                                                                 
Total params: 820,739
Trainable params: 820,739
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(xTrain,yTrain,epochs=15,batch_size=32,validation_data=(xTest,yTest))
acr = str(round(model.evaluate(xTest,yTest)[1],4)).replace("0.","")
model.save(f"TG14_{acr}.h5")

Epoch 1/15
5167/5167 [==============================] - 67s 13ms/step - loss: 0.7490 - accuracy: 0.6682 - val_loss: 0.6585 - val_accuracy: 0.7154
Epoch 2/15
5167/5167 [==============================] - 70s 14ms/step - loss: 0.5348 - accuracy: 0.7802 - val_loss: 0.4793 - val_accuracy: 0.8138
Epoch 3/15
5167/5167 [==============================] - 65s 13ms/step - loss: 0.3416 - accuracy: 0.8727 - val_loss: 0.3484 - val_accuracy: 0.8772
Epoch 4/15
5167/5167 [==============================] - 62s 12ms/step - loss: 0.2282 - accuracy: 0.9215 - val_loss: 0.2891 - val_accuracy: 0.9092
Epoch 5/15
5167/5167 [==============================] - 62s 12ms/step - loss: 0.1704 - accuracy: 0.9443 - val_loss: 0.2533 - val_accuracy: 0.9278
Epoch 6/15
5167/5167 [==============================] - 63s 12ms/step - loss: 0.1392 - accuracy: 0.9556 - val_loss: 0.2382 - val_accuracy: 0.9341
Epoch 7/15
5167/5167 [==============================] - 65s 13ms/step - loss: 0.1210 - accuracy: 0.9624 - val_loss: 0.2265 -

In [28]:
def process_for_prediction(data,index):
        i = index
        if "symbol" in data.columns:
              data.drop("symbol",axis=1,inplace=True) 
        if "datetime" in data.columns:
              data.drop("datetime",axis=1,inplace=True)
        if "Adj Close" in data.columns:
              data.drop("Adj Close",axis=1,inplace=True)
        data = np.array(data)
        grow = []
        ggggrow = []
        for x in range(1, 31):
            grow.append([data[i-x][0] - data[i-(1+x)][0]] )
            ggggrow.append([data[i-x][3] - data[i-(1+x)][3]] )

        arr = np.array(grow).flatten()
        arr4 = np.array(ggggrow).flatten()

        arr = np.concatenate((arr,arr4), axis=0).reshape(-1,1)
        arr = scaler(arr.reshape(-1, 1))

        high = 0
        low = data[i][0] * 10000
        for x in range(i-5, i):
          if data[x][0] > high:
              high = data[x][0]
          if data[x][3] < low:
              low = data[x][3]


        diff = high - low
        mean_of_last = (data[i-1][0] + data[i-1][3])/2
        print(f"Diff={diff} high={high} low={low} mean={mean_of_last} estimated_high=>{mean_of_last+diff} estimated_low<={mean_of_last-diff}")
        return arr
def make_prediction_for_yf(symbol,period,timeframe,index):
    raw_data = process_for_prediction(yf.download(symbol,period=period,interval=timeframe),index)
    return f"YF  : {model.predict(np.array(raw_data).reshape(1,-1))}"
def make_prediction_for_tv(symbol,exchange,timeframe,tindex):
   tv = TvDatafeed()
   raw_data = process_for_prediction(tv.get_hist(symbol=symbol,exchange=exchange,interval=timeframe,n_bars=100),tindex)
   return  f"TVB : {model.predict(np.array(raw_data).reshape(1,-1))}"
def make_prediction(ysymbol,period,timeframe,tsymbol,texchange,ttimeframe,tindex,index):
  print(make_prediction_for_yf(ysymbol,period,timeframe,index))
  print(make_prediction_for_tv(tsymbol,texchange,ttimeframe,tindex))

In [30]:
make_prediction("btc-usd","max","1mo","btcusdt","binance",Interval.in_monthly,-1,-2)

[*********************100%***********************]  1 of 1 completed


Diff=1959.822265625 high=45554.1640625 low=37714.875 mean=41634.51953125 estimated_high=>43594.341796875 estimated_low<=39674.697265625
YF  : [[8.1148696e-01 2.5304822e-05 9.9621880e-01]]
Diff=1969.887499999999 high=45510.35 low=37630.8 mean=41570.575 estimated_high=>43540.462499999994 estimated_low<=39600.6875
TVB : [[8.0854034e-01 2.6155134e-05 9.9618745e-01]]


In [15]:
yf.download("btc-usd",period="10d",interval="1h")

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
2022-05-09 00:00:00+00:00,34060.015625,34191.234375,34039.796875,34169.183594,34169.183594,166002688
2022-05-09 01:00:00+00:00,34162.781250,34222.074219,34083.222656,34088.246094,34088.246094,0
2022-05-09 02:00:00+00:00,34087.449219,34087.449219,33393.613281,33422.035156,33422.035156,183775232
2022-05-09 03:00:00+00:00,33473.324219,33771.390625,33447.359375,33611.773438,33611.773438,391393280
2022-05-09 04:00:00+00:00,33582.140625,33796.695312,33558.632812,33673.382812,33673.382812,0
...,...,...,...,...,...,...
2022-05-18 07:00:00+00:00,29784.169922,29860.083984,29737.574219,29860.083984,29860.083984,319932416
2022-05-18 08:00:00+00:00,29868.677734,29908.021484,29766.435547,29848.974609,29848.974609,975056896
2022-05-18 09:00:00+00:00,29852.814453,29948.996094,29793.697266,29919.140625,29919.140625,440352768
2022-05-18 10:00:00+00:00,29917.369141,30016.177734,29915.722656,29915.722656,29915.722656,341360640
